In [114]:
'''
    This jupyter notebook scrapes Airbnb data in 7 cities using basic web scraping methods. 
    
    Much of this code was referenced from https://github.com/rubenkerkhofs/Airbnb-scraping.
    Thanks Ruben Kerkhofs. 
    His tutorial is available here: https://medium.com/analytics-vidhya/scraping-airbnb-fe1e895bd925
'''

from bs4 import BeautifulSoup 
# The following packages will also be used in this tutorial
import pandas as pd # All database operationss
import numpy as np  # Numerical operations
import time         # Tracking time
import requests     # HTTP requests
import urllib.request
import re           # String manipulation
import pickle
from tqdm import trange, tqdm
import matplotlib.pyplot as plt

#web scraping with selenium
from selenium import webdriver
import chromedriver_binary  # Adds chromedriver binary to path
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException




In [137]:
"""
    Getting started
"""
def getPage(url):
    ''' returns a soup object that contains all the information 
    of a certain webpage'''
    time.sleep(5)
    result = requests.get(url)
    content = result.content
    return BeautifulSoup(content, features = "lxml")

    
def getRoomClasses(soupPage):
    ''' This function returns all the listings that can 
    be found on the page in a list.'''
    rooms = soupPage.findAll("div", {"class": "_8ssblpx"})
    result = []
    for room in rooms:
        result.append(room)
    return result

def getListingLink(listing):
    ''' This function returns the link of the listing'''
    return "http://airbnb.com" + listing.find("a")["href"]

def getListingTitle(listing):
    ''' This function returns the title of the listing'''
    return listing.find("meta")["content"]

def getTopRow(listing):
    ''' Returns the top row of listing information'''
    return listing.find("div", {"class": "_167qordg"}).text

def getRoomInfo(listing):
    ''' Returns the guest information'''
    return listing.find("div", {"class":"_kqh46o"}).text

def getListingPrice(listing):
    ''' Returns the price'''
    return listing.find("div", {"class":"_1fwiw8gv"}).text

def getImageLink(listing):
    ''' Returns the img link'''
    return listing.find('img')['data-original-uri']

def extractInformation(soupPage):
    ''' Takes all the information of a single page (thus multiple listings) and
    summarizes it in a dataframe'''
    listings = getRoomClasses(soupPage)
    titles, links, toprows, basicfacilitiess, roominfos, prices, images = [], [], [], [], [], [], []
    for listing in listings:
        titles.append(getListingTitle(listing))
        links.append(getListingLink(listing))
        toprows.append(getTopRow(listing))
        roominfos.append(getRoomInfo(listing))
        prices.append(getListingPrice(listing))
        images.append(getImageLink(listing))

    dictionary = {"title": titles, "toprow": toprows, "roominfo": roominfos, "price": prices,"link": links,"image":images}
    return pd.DataFrame(dictionary)
       
'''
    Scraping all listings for a given city
'''
def findNextPage(soupPage):
    ''' Finds the next page with listings if it exists '''
    try:
        nextpage = "https://airbnb.com" + soupPage.find("li", {"class": "_i66xk8d"}).find("a")["href"]
    except:
        nextpage = "no next page"
    return nextpage

def getPages(url):
    ''' This function returns all the links to the pages containing 
    listings for one particular city '''
    result = []
    while url != "no next page": 
        page = getPage(url)
        result = result + [page]
        url = findNextPage(page)
    return result

def extractPages(url):
    ''' This function outputs a dataframe that contains all information of a particular
    city. It thus contains information of multiple listings coming from multiple pages.'''
    pages = getPages(url)
    # Do for the first element to initialize the dataframe
    df = extractInformation(pages[0])
    # Loop over all other elements of the dataframe
    for pagenumber in trange(1, len(pages)):
        df = df.append(extractInformation(pages[pagenumber]))
    return df

'''
    Clean functions basic data frame extracted using only beautifulsoup
'''
def cleanTitle(df):
    df.loc[:, "name"] = df["title"].str.split(" null ", n = 0, expand = True)[0].str.replace("-", "")
    df.loc[:, "location"] = df["title"].str.split(" null ", n = 0, expand = True)[1].str.replace("-", "").str.strip()
    return df.drop("title", axis = 1)

def cleanTopRow(df):
    df.loc[:, 'roomtype'] = df["toprow"].str.split(" in ", n = 0, expand = True)[0] 
    df.loc[:, 'detailed_location'] = df["toprow"].str.split(" in ", n = 0, expand = True)[1] 
    return df.drop("toprow", axis = 1)

def cleanRoomInfo(df):
    df.loc[:, "bedrooms"] = df.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[1]
    df.loc[:, "beds"] = df.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[2].str.replace(" bed", "").str.replace("s", "")
    df.loc[:, "beds"] = pd.to_numeric(df.beds, errors = 'coerce')
    df.loc[:, "bedrooms"] = pd.to_numeric(df.bedrooms.str.split(" ", n = 0, expand = True)[0], errors = "ignore")
    return df.drop("roominfo", axis = 1)

def cleanPrice(df):
    price_list = []
    #convert price to number, takes the discounted price.
    #requests accessed airbnb from Singapore, prices shown in SGD converted to USD
    #exchange rate as of 26/9/2020
    for index, row in df.iterrows():
        if "Discounted" in row['price']:
            row['price'] = row['price'].split("Discounted")[1]

            row['price'] = round(float(re.sub('[^0-9]','', row['price'])) * 0.73)
        else:
            row['price'] = re.sub('[^0-9]','', row['price'])
            row['price'] = round(float(re.sub('[^0-9]','', row['price'])) * 0.73) 

        price_list.append(row['price'])

    df['listed_price'] = price_list
    df['price'] = round(df['listed_price'] / df['beds'])
    return df


def clean(df):
    df = cleanTitle(df)
    df = cleanTopRow(df)
    df = cleanRoomInfo(df)
    df = cleanPrice(df)
    return df


In [138]:
'''
Details on search queries to Airbnb
- Up to 3000 usd to remove stays that Minervans are unlikely to rent anyways
- At least 1 bed (this is how price is calculated, since the price is split by the number of beds in the stay)
- Has Wifi + Kitchen
- Search Jan 1st - April 30th 2021. 
- Specific query terms in the urls below
'''
cities = {
    "San Francisco" : "https://www.airbnb.com.sg/s/San-Francisco--CA--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2021-01-01&checkout=2021-04-30&source=structured_search_input_header&search_type=autocomplete_click&amenities%5B%5D=4&amenities%5B%5D=8&price_max=4134&min_beds=1&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Hotel%20room&room_types%5B%5D=Private%20room&query=San%20Francisco%2C%20CA%2C%20United%20States",
    "Seoul": "https://www.airbnb.com.sg/s/Seoul--South-Korea/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2021-01-01&checkout=2021-04-30&source=structured_search_input_header&search_type=autocomplete_click&amenities%5B%5D=4&amenities%5B%5D=8&price_max=4134&min_beds=1&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Hotel%20room&room_types%5B%5D=Private%20room&query=Seoul%2C%20South%20Korea&place_id=ChIJzWXFYYuifDUR64Pq5LTtioU",
    "Hyderabad": "https://www.airbnb.com.sg/s/Hyderabad--Telangana/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2021-01-01&checkout=2021-04-30&source=structured_search_input_header&search_type=search_query&amenities%5B%5D=4&amenities%5B%5D=8&price_max=4134&min_beds=1&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Private%20room",
    "Berlin": "https://www.airbnb.com.sg/s/berlin/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2021-01-01&checkout=2021-04-30&source=structured_search_input_header&search_type=search_query&amenities%5B%5D=4&amenities%5B%5D=8&price_max=4134&min_beds=1&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Hotel%20room&room_types%5B%5D=Private%20room",
    "Buenos Aires": "https://www.airbnb.com.sg/s/Buenos-Aires--Ciudad-Aut%C3%B3noma-de-Buenos-Aires--Argentina/homes?refinement_paths%5B%5D=%2Fhomes&checkin=2021-01-01&checkout=2021-04-30&source=structured_search_input_header&search_type=autocomplete_click&amenities%5B%5D=4&amenities%5B%5D=8&price_max=4134&min_beds=1&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Hotel%20room&room_types%5B%5D=Private%20room&click_referer=t%3ASEE_ALL%7Csid%3Aaf552623-f9f0-4be8-98b7-75f840406057%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&tab_id=home_tab&query=Buenos%20Aires%2C%20Ciudad%20Aut%C3%B3noma%20de%20Buenos%20Aires%2C%20Argentina&place_id=ChIJvQz5TjvKvJURh47oiC6Bs6A",
    "Taipei": "https://www.airbnb.com.sg/s/Taipei-City--Taiwan/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2021-01-01&checkout=2021-04-30&source=structured_search_input_header&search_type=autocomplete_click&amenities%5B%5D=4&amenities%5B%5D=8&price_max=4134&min_beds=1&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Hotel%20room&room_types%5B%5D=Private%20room&query=Taipei%20City%2C%20Taiwan&place_id=ChIJi73bYWusQjQRgqQGXK260bw",
    "London": "https://www.airbnb.com.sg/s/London--United-Kingdom/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2021-01-01&checkout=2021-04-30&source=structured_search_input_header&search_type=autocomplete_click&amenities%5B%5D=4&amenities%5B%5D=8&price_max=4134&min_beds=1&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Hotel%20room&room_types%5B%5D=Private%20room&query=London%2C%20United%20Kingdom&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI"
}


city_dfs = []
for key in tqdm(cities):
    url = cities[key]
    
    df = extractPages(url)
    df['city'] = key
    city_dfs.append(clean(df))
    print(f'Done with {key}')




  0%|          | 0/7 [00:00<?, ?it/s]



  0%|          | 0/14 [00:00<?, ?it/s]



 21%|██▏       | 3/14 [00:00<00:00, 26.43it/s]



 43%|████▎     | 6/14 [00:00<00:00, 26.19it/s]



 64%|██████▍   | 9/14 [00:00<00:00, 26.25it/s]



100%|██████████| 14/14 [00:00<00:00, 26.24it/s]



 14%|█▍        | 1/7 [01:58<11:53, 118.93s/it]

Done with San Francisco






  0%|          | 0/14 [00:00<?, ?it/s]



 21%|██▏       | 3/14 [00:00<00:00, 26.23it/s]



 43%|████▎     | 6/14 [00:00<00:00, 26.26it/s]



 64%|██████▍   | 9/14 [00:00<00:00, 26.49it/s]



100%|██████████| 14/14 [00:00<00:00, 26.21it/s]



 29%|██▊       | 2/7 [04:00<09:58, 119.71s/it]

Done with Seoul






  0%|          | 0/8 [00:00<?, ?it/s]



 38%|███▊      | 3/8 [00:00<00:00, 27.87it/s]



100%|██████████| 8/8 [00:00<00:00, 30.43it/s]



 43%|████▎     | 3/7 [05:11<07:00, 105.12s/it]

...

Done with Buenos Aires






  0%|          | 0/14 [00:00<?, ?it/s]



 21%|██▏       | 3/14 [00:00<00:00, 27.08it/s]



 43%|████▎     | 6/14 [00:00<00:00, 27.12it/s]



 64%|██████▍   | 9/14 [00:00<00:00, 26.96it/s]



100%|██████████| 14/14 [00:00<00:00, 27.10it/s]



 86%|████████▌ | 6/7 [11:09<01:54, 114.30s/it]

Done with Taipei






  0%|          | 0/14 [00:00<?, ?it/s]



 21%|██▏       | 3/14 [00:00<00:00, 27.08it/s]



 43%|████▎     | 6/14 [00:00<00:00, 27.17it/s]



 64%|██████▍   | 9/14 [00:00<00:00, 27.21it/s]



100%|██████████| 14/14 [00:00<00:00, 27.19it/s]



100%|██████████| 7/7 [13:11<00:00, 113.12s/it]

Done with London


In [139]:
import pickle
pickle.dump(city_dfs, open( "data.p", "wb" ) )


# Full Data Set For Viz

In [293]:
city_dfs = pd.read_pickle("./data.p")
price_df = pd.concat(city_dfs)
price_df = price_df[price_df.beds < 5]

price_df = price_df.drop_duplicates(subset=['name'])
only_prices_df = price_df.filter(['price', 'city'])


In [294]:
only_prices_df.to_csv("dev/prices_data.csv",index=False)

# Get Cheaper Than Minerva Data!

In [142]:
full_df = pd.concat(city_dfs)

#clean hostels ugh
full_df = full_df[full_df.beds < 5]

#filter out more exp than Minerva
full_df = full_df[full_df.price < 750]


In [173]:
def getScoreAndFullAmenities(df):
    rating = []
    review_num = []
    full_rating = []
    stuff = []
    
    for url in tqdm(df['link']):
        print(url)
        
        time.sleep(2)
        
        driver = webdriver.Chrome(ChromeDriverManager().install())

        driver.get(url)
        
        #scores
        try:
            wait = WebDriverWait(driver, 5)
            wait.until(EC.presence_of_element_located((By.CLASS_NAME,"_1jpdmc0")))
            soup = BeautifulSoup(driver.page_source, 'lxml')

            #find scores
            score = soup.find("span", {"class":"_1jpdmc0"}).text
            reviews = soup.find("span", {"class":"_1sqnphj"}).text
            full = f'{score} out of 5 stars from {reviews[1:-1]} reviews'

            rating.append(score)
            review_num.append(reviews[1:-1])
            full_rating.append(full)

        except TimeoutException as ex:
            rating.append(None)
            review_num.append(0)
            full_rating.append("No Reviews Yet!")


        try:
            waitMore = WebDriverWait(driver, 5)
            waitMore.until(EC.presence_of_element_located((By.CLASS_NAME,"_1nlbjeu")))
            soup = BeautifulSoup(driver.page_source, 'lxml')

            #find full amenities
            features = [a.text for a in soup.findAll("div", {"class":"_1nlbjeu"}) if "Unavailable" not in a.text] 
            stuff.append(features)

        except TimeoutException as ex:
            stuff.append([Wifi,Kitchen])   


        driver.quit()
        
        
    return rating, review_num, full_rating, stuff
        
        

        
        

In [146]:
rating, review_num, full_rating, stuff = getScoreAndFullAmenities(full_df)

http://airbnb.com/rooms/45224017?check_in=2021-01-01&check_out=2021-04-30&previous_page_section_name=1000&display_extensions%5B%5D=DATED_LTS&display_extensions%5B%5D=MONTHLY_STAYS&federated_search_id=68d4a45d-fdd3-44d9-b314-b4b15a9e7215


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/jasenlo/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


http://airbnb.com/rooms/20367796?check_in=2021-01-01&check_out=2021-04-30&previous_page_section_name=1000&display_extensions%5B%5D=MONTHLY_STAYS&display_extensions%5B%5D=DATED_LTS&federated_search_id=bbef1221-b402-4764-91ff-8c4708f93713


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/jasenlo/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


...

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/jasenlo/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


 
http://airbnb.com/rooms/23068010?check_in=2021-01-01&check_out=2021-04-30&previous_page_section_name=1000&display_extensions%5B%5D=DATED_LTS&display_extensions%5B%5D=MONTHLY_STAYS&federated_search_id=6be241a6-857f-4407-9890-8af26367d338


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/jasenlo/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


 
http://airbnb.com/rooms/29392703?check_in=2021-01-01&check_out=2021-04-30&previous_page_section_name=1000&display_extensions%5B%5D=DATED_LTS&display_extensions%5B%5D=MONTHLY_STAYS&federated_search_id=6be241a6-857f-4407-9890-8af26367d338


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/jasenlo/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [190]:
#full_df['rating'] = rating
#full_df['review_number'] = review_num



#full_df['full_rating'] = full_rating
#full_df['amenities'] = stuff



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [183]:
#save full dataset as a pickle!
pickle.dump(full_df, open( "full_data.p", "wb" ) )

In [314]:
full_df = pd.read_pickle("./full_data.p")
full_df['review_number'] = full_df['review_number'].astype(float)
full_df = full_df[full_df.review_number > 0]

amenities = []
image_small = []
full_df['rating'] = full_df['rating'].astype(float)
for index, row in full_df.iterrows():
    string = ', '.join([str(elem) for elem in row['amenities']])
    string = string.replace('Essentials', 'Toilet paper, Soap, Towels, Pillows, Bed linens')
    amenities.append(string)
    
    img = row['image'][:-3] + str(240)
    image_small.append(img)
    
    
    
full_df['amenities'] = amenities 
full_df['image'] = image_small
full_df = full_df.drop_duplicates(subset=['name',"full_rating"])
full_df = full_df.sort_values(by='price', ascending=False)
full_df = full_df.reset_index(drop=True)


In [317]:
full_df.to_csv("dev/full_data.csv",index=False)

In [318]:
full_df

,price,link,image,city,name,location,roomtype,detailed_location,bedrooms,beds,listed_price,rating,full_rating,amenities,review_number
0,748.0,http://airbnb.com/rooms/5025066?check_in=2021-...,https://a0.muscache.com/im/pictures/62908502/8...,Berlin,Room in green area close to Berlin,Falkensee,Private room,Falkensee,1,1,748,4.97,4.97 out of 5 stars from 78 reviews,"Wifi, Kitchen, Free parking on premises, Pool,...",78.0
1,745.0,http://airbnb.com/rooms/12148040?check_in=2021...,https://a0.muscache.com/im/pictures/b42a79a5-b...,Berlin,"Nice, light and friendly room",Berlin,Private room,Schöneberg,1,1,745,4.59,4.59 out of 5 stars from 54 reviews,"Wifi, Kitchen, Dryer, Cable TV, Iron, Laptop-f...",54.0
2,745.0,http://airbnb.com/rooms/29080921?check_in=2021...,https://a0.muscache.com/im/pictures/c4d2a26c-b...,Seoul,"♥Seoul stay♥[Metro 5min] Myeongdong,Dongdaemun","Suyudong, Gangbukgu",Entire flat,Gangbuk,1,1,745,4.94,4.94 out of 5 stars from 32 reviews,"Kitchen, Wifi, Iron, Hair dryer, Cable TV, Han...",32.0
3,743.0,http://airbnb.com/rooms/39928292?check_in=2021...,https://a0.muscache.com/im/pictures/047462a6-1...,Seoul,"♥Cozy♥Clean서울penthouse 보문역도보3분,고려대학교,성신여대,동대문,종로",Seongbukgu,Entire loft,Seongbuk,Studio,1,743,4.67,4.67 out of 5 stars from 18 reviews,"Kitchen, Wifi, Dryer, Hair dryer, Cable TV, Ha...",18.0
4,743.0,http://airbnb.com/rooms/19099053?check_in=2021...,https://a0.muscache.com/im/pictures/98ed306d-2...,Berlin,Familyfriendly apartment in the center of Berlin,Berlin,Entire flat,Friedrichshain,1,2,1486,4.85,4.85 out of 5 stars from 151 reviews,"Wifi, Kitchen, Cable TV, Iron, Laptop-friendly...",151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,88.0,http://airbnb.com/rooms/22821431?check_in=2021...,https://a0.muscache.com/im/pictures/ad66ebbf-4...,Hyderabad,Fully Furnished Flatwith/without AC. Reburbis...,Hyderabad,Entire flat,Hyderabad,2,4,353,4.37,4.37 out of 5 stars from 27 reviews,"Lift, Kitchen, Wifi, Free parking on premises,...",27.0
628,88.0,http://airbnb.com/rooms/6029502?check_in=2021-...,https://a0.muscache.com/im/pictures/75091362/7...,Buenos Aires,New Apartments in San Telmo,Buenos Aires,Entire flat,San Telmo,Studio,3,265,4.73,4.73 out of 5 stars from 96 reviews,"Lift, Wifi, Kitchen, Pool, Iron, Cable TV, Lap...",96.0
629,81.0,http://airbnb.com/rooms/42602509?check_in=2021...,https://a0.muscache.com/im/pictures/9a2a24fc-d...,Hyderabad,The Happy Place,Secunderabad,Entire house,Secunderabad,2,4,323,5.00,5.0 out of 5 stars from 11 reviews,"Kitchen, Wifi, Hair dryer, Cable TV, Iron, Han...",11.0
630,71.0,http://airbnb.com/rooms/39198067?check_in=2021...,https://a0.muscache.com/im/pictures/5edf5dbc-5...,Buenos Aires,"Renovated house, in the heart of San Telmo",San Telmo,Private room,San Telmo,1,4,285,5.00,5.0 out of 5 stars from 4 reviews,"Wifi, Kitchen, Iron, Laptop-friendly workspace...",4.0
